<a href="https://colab.research.google.com/github/rpandya5/gaitanalysis/blob/main/baseline%20model/SVM_Test_Data(Phone_Collected).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Window Sampling and Feature Extraction done on the Phone Collected SVM Data and saving it according to Baseline SVM Requirements**







In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
class DataProcessing:
  def __init__(self, window_size = 400, window_overlap = 200):

    #Features we extract from the windows
    self.features = ['X_Acc_Mean', 'X_Acc_Max', 'X_Acc_Std', 'X_Acc_Min',
                     'X_Acc_IQR', 'X_Acc_Skew', 'X_Acc_Kurtosis', 'X_Acc_Energy', 'X_Acc_Jerk',
                     'Y_Acc_Mean', 'Y_Acc_Max', 'Y_Acc_Std', 'Y_Acc_Min',
                     'Y_Acc_IQR', 'Y_Acc_Skew', 'Y_Acc_Kurtosis', 'Y_Acc_Energy', 'Y_Acc_Jerk',
                     'Z_Acc_Mean', 'Z_Acc_Max', 'Z_Acc_Std', 'Z_Acc_Min',
                     'Z_Acc_IQR', 'Z_Acc_Skew', 'Z_Acc_Kurtosis', 'Z_Acc_Energy', 'Z_Acc_Jerk',
                     'XY_Acc_Cov', 'X_Gyro_Mean', 'X_Gyro_Max', 'X_Gyro_Std', 'X_Gyro_Min',
                     'X_Gyro_IQR', 'X_Gyro_Skew', 'X_Gyro_Kurtosis', 'X_Gyro_Energy', 'X_Gyro_Jerk',
                     'Y_Gyro_Mean', 'Y_Gyro_Max', 'Y_Gyro_Std', 'Y_Gyro_Min',
                     'Y_Gyro_IQR', 'Y_Gyro_Skew', 'Y_Gyro_Kurtosis', 'Y_Gyro_Energy', 'Y_Gyro_Jerk',
                     'Z_Acc_Mean', 'Z_Gyro_Max', 'Z_Gyro_Std', 'Z_Gyro_Min',
                     'Z_Gyro_IQR', 'Z_Gyro_Skew', 'Z_Gyro_Kurtosis', 'Z_Gyro_Energy', 'Z_Gyro_Jerk',
                     'XY_Gyro_Cov']

    self.window_size = window_size # window size is 2 seconds
    self.window_overlap = window_overlap # Overlap is 1 second

  #Feature Extraction Done by Anya and Nick
  def getcov(self, x_list_1, x_bar ,y_list, y_bar):
    """ Calculates the covariance of x and y in the selected window.

    Parameters:
    -----------
        x_list_1 : list
        x_bar :
          Average of x list
        y_list : list
        y_bar : float
          Average of y list

    Returns:
    --------
        covariace of x and y : float

    """
    cov = []
    for i in range(len(x_list_1)):
        cov.append((x_list_1[i] - x_bar) * (y_list[i] - y_bar))
    return sum(cov) / len(x_list_1) - 1

  def jerk(self, x_list, t_step):
    """ Calculated jerk, or the rate of change of the acceleration.

    Parameters:
    -----------
    x_list : list
    t_step : float

    Returns:
    --------
    jerk_list : list

    """
    jerk_list = []
    for i in range(len(x_list)-1):
        jerk_list.append((x_list[i+1] - x_list[i]) / t_step)

    jerk_list.append(jerk_list[-1])
    return sum(jerk_list) / len(jerk_list)

  def energy(self, x_list):
    """ Avergaged energy of the corresponding window

    Parameters:
    -----------
    x_list : list

    Returns:
    --------
    energy : float

    """
    sum = 0
    for i in range(len(x_list)):
        sum += x_list[i]**2
    return sum / len(x_list)


  #Nick add your functions here (also add self as the first parameter)
  def butter_lowpass_filter(self, data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

  def getmean(self,df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.mean()

  def getstd(self,df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.std()

  def getmin(self, df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.min()

  def getmax(self, df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.max()

  def getmedian(self, df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.median()

  def getskewness(self, df, col):
    """if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.skew()

  def getkurtosis(self, df, col):
    """ if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.kurtosis()

  def getiqr(self, df, col):
    """ if col in (0,1,2,6,7,8):
      x = butter_lowpass_filter(df[col], 10, 100)
    else:"""
    x = df[col]
    return x.quantile(0.75) - x.quantile(0.25)
  #Window Sampling done by Krish
  def window_sample(self, file_df):
    """
    Description: Converts sensor readings of 15 seconds (200 Hz) into windows

    Inputs:   dir- path to the directory where the file is stored
              file- Name of the file with the sensor data

    Outputs:  DataFrame with the windows and features extracted
    """

    #Window size for activities is 2 seconds with a one second overlap
    window_x = 1
    new_df = []
    for i in range(0, len(file_df)-self.window_overlap, self.window_overlap):
      window = file_df.iloc[i:i+self.window_size:]
      sampled = []

      #Calling all functions for the specific window
      sampled.append(self.getmean(window, 'X_Acc_1'))
      sampled.append(self.getmax(window, 'X_Acc_1'))
      sampled.append(self.getstd(window, 'X_Acc_1'))
      sampled.append(self.getmin(window, 'X_Acc_1'))
      sampled.append(self.getiqr(window, 'X_Acc_1'))
      sampled.append(self.getskewness(window, 'X_Acc_1'))
      sampled.append(self.getkurtosis(window, 'X_Acc_1'))
      sampled.append(self.energy(window['X_Acc_1'].tolist()))
      sampled.append(self.jerk(window['X_Acc_1'].tolist(), (2/self.window_size)))

      sampled.append(self.getmean(window, 'Y_Acc_1'))
      sampled.append(self.getmax(window, 'Y_Acc_1'))
      sampled.append(self.getstd(window, 'Y_Acc_1'))
      sampled.append(self.getmin(window, 'Y_Acc_1'))
      sampled.append(self.getiqr(window, 'Y_Acc_1'))
      sampled.append(self.getskewness(window, 'Y_Acc_1'))
      sampled.append(self.getkurtosis(window, 'Y_Acc_1'))
      sampled.append(self.energy(window['Y_Acc_1'].tolist()))
      sampled.append(self.jerk(window['Y_Acc_1'].tolist(), (2/self.window_size)))

      sampled.append(self.getmean(window, 'Z_Acc_1'))
      sampled.append(self.getmax(window, 'Z_Acc_1'))
      sampled.append(self.getstd(window, 'Z_Acc_1'))
      sampled.append(self.getmin(window, 'Z_Acc_1'))
      sampled.append(self.getiqr(window, 'Z_Acc_1'))
      sampled.append(self.getskewness(window, 'Z_Acc_1'))
      sampled.append(self.getkurtosis(window, 'Z_Acc_1'))
      sampled.append(self.energy(window['Z_Acc_1'].tolist()))
      sampled.append(self.jerk(window['Z_Acc_1'].tolist(), (2/self.window_size)))
      sampled.append(self.getcov(window['X_Acc_1'].tolist(), self.getmean(window, 'X_Acc_1'), window['Y_Acc_1'].tolist(), self.getmean(window, 'Y_Acc_1')))


      sampled.append(self.getmean(window, 'X_Gyro'))
      sampled.append(self.getmax(window, 'X_Gyro'))
      sampled.append(self.getstd(window, 'X_Gyro'))
      sampled.append(self.getmin(window, 'X_Gyro'))
      sampled.append(self.getiqr(window, 'X_Gyro'))
      sampled.append(self.getskewness(window, 'X_Gyro'))
      sampled.append(self.getkurtosis(window, 'X_Gyro'))
      sampled.append(self.energy(window['X_Gyro'].tolist()))
      sampled.append(self.jerk(window['X_Gyro'].tolist(), (2/self.window_size)))

      sampled.append(self.getmean(window, 'Y_Gyro'))
      sampled.append(self.getmax(window, 'Y_Gyro'))
      sampled.append(self.getstd(window, 'Y_Gyro'))
      sampled.append(self.getmin(window, 'Y_Gyro'))
      sampled.append(self.getiqr(window, 'Y_Gyro'))
      sampled.append(self.getskewness(window, 'Y_Gyro'))
      sampled.append(self.getkurtosis(window, 'Y_Gyro'))
      sampled.append(self.energy(window['Y_Gyro'].tolist()))
      sampled.append(self.jerk(window['Y_Gyro'].tolist(), (2/self.window_size)))

      sampled.append(self.getmean(window, 'Z_Gyro'))
      sampled.append(self.getmax(window, 'Z_Gyro'))
      sampled.append(self.getstd(window, 'Z_Gyro'))
      sampled.append(self.getmin(window, 'Z_Gyro'))
      sampled.append(self.getiqr(window, 'Z_Gyro'))
      sampled.append(self.getskewness(window, 'Z_Gyro'))
      sampled.append(self.getkurtosis(window, 'Z_Gyro'))
      sampled.append(self.energy(window['Z_Gyro'].tolist()))
      sampled.append(self.jerk(window['Z_Gyro'].tolist(), (2/self.window_size)))
      sampled.append(self.getcov(window['X_Gyro'].tolist(), self.getmean(window, 'X_Gyro'), window['Y_Gyro'].tolist(), self.getmean(window, 'Y_Gyro')))

      new_df.append(sampled)
      window_x += 1

    #Returns the created DataFrame
    return pd.DataFrame(new_df, columns=self.features)

In [ ]:
phone_data = []
phone_labels = []
sampler = DataProcessing(20, 10)
for dir, _, files in os.walk('/content/gdrive/MyDrive/Phone Collected Data'):
  for file in files:
    if file[0] == 'p':
      file_df = pd.read_csv(os.path.join(dir, file))
      cols = file_df.columns
      file_df['Timestamp'] = file_df['seconds_elapsed']
      file_df['X_Acc_1'] = file_df['accelerometer_x']*-1
      file_df['Y_Acc_1'] = file_df['accelerometer_y']*-1
      file_df['Z_Acc_1'] = file_df['accelerometer_z']*-1
      file_df['X_Gyro'] = file_df['gyroscope_x']*(-180/3.14)
      file_df['Y_Gyro'] = file_df['gyroscope_y']*(-180/3.14)
      file_df['Z_Gyro'] = file_df['gyroscope_z']*(-180/3.14)
      file_df = file_df.drop(cols, axis=1)
      file_df = file_df[20:170]
      file_sampled = sampler.window_sample(file_df)
      phone_data.append(file_sampled)
      phone_labels.append(1)
    else:
      file_df = pd.read_csv(os.path.join(dir, file))
      cols = file_df.columns
      file_df['Timestamp'] = file_df['seconds_elapsed']
      file_df['X_Acc_1'] = file_df['accelerometer_x']*-1
      file_df['Y_Acc_1'] = file_df['accelerometer_y']*-1
      file_df['Z_Acc_1'] = file_df['accelerometer_z']*-1
      file_df['X_Gyro'] = file_df['gyroscope_x']*(-180/3.14)
      file_df['Y_Gyro'] = file_df['gyroscope_y']*(-180/3.14)
      file_df['Z_Gyro'] = file_df['gyroscope_z']*(-180/3.14)
      file_df = file_df.drop(cols, axis=1)
      if len(file_df)<150:
        cols = file_df.columns
        file_np = np.array(file_df)
        file_df = pd.DataFrame(np.array(np.vstack((file_np, np.tile(file_np[-1], (150-len(file_df), 1))))), columns=cols)
      elif len(file_df)>150:
        file_df = file_df[:150]
      file_sampled = sampler.window_sample(file_df)
      phone_data.append(file_sampled)
      phone_labels.append(0)

In [ ]:
np.array(phone_data).shape

(97, 14, 56)

In [ ]:
np.array(phone_labels).shape

(97,)

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/Copy of Baseline SVM Data.csv')

In [ ]:
train_data.columns[1:]

Index(['X_Acc_Mean_1', 'X_Acc_Max_1', 'X_Acc_Std_1', 'X_Acc_Min_1',
       'X_Acc_IQR_1', 'X_Acc_Skew_1', 'X_Acc_Kurtosis_1', 'X_Acc_Energy_1',
       'X_Acc_Jerk_1', 'Y_Acc_Mean_1',
       ...
       'Z_Gyro_Max_14', 'Z_Gyro_Std_14', 'Z_Gyro_Min_14', 'Z_Gyro_IQR_14',
       'Z_Gyro_Skew_14', 'Z_Gyro_Kurtosis_14', 'Z_Gyro_Energy_14',
       'Z_Gyro_Jerk_14', 'XY_Gyro_Cov_14', 'Label'],
      dtype='object', length=785)

In [ ]:
phone_data_reshaped = (np.array(phone_data)).reshape((97, 14*56))

In [ ]:
final_test = pd.DataFrame(np.hstack((phone_data_reshaped, np.array(phone_labels).reshape(97, 1))), columns=train_data.columns[1:])

In [ ]:
final_test

,X_Acc_Mean_1,X_Acc_Max_1,X_Acc_Std_1,X_Acc_Min_1,X_Acc_IQR_1,X_Acc_Skew_1,X_Acc_Kurtosis_1,X_Acc_Energy_1,X_Acc_Jerk_1,Y_Acc_Mean_1,...,Z_Gyro_Max_14,Z_Gyro_Std_14,Z_Gyro_Min_14,Z_Gyro_IQR_14,Z_Gyro_Skew_14,Z_Gyro_Kurtosis_14,Z_Gyro_Energy_14,Z_Gyro_Jerk_14,XY_Gyro_Cov_14,Label
0,-0.206647,1.235590,0.999630,-2.080345,0.987887,-0.562808,-0.418538,0.991999,0.680692,-0.248574,...,63.503197,19.310671,-28.866841,24.545379,1.035257,3.091757,397.173339,-11.280439,559.588747,0.0
1,0.880420,4.702990,1.308016,-1.302499,1.014827,1.050809,2.990133,2.400499,4.331345,0.184174,...,114.463318,55.505998,-135.713981,46.990146,-0.429392,2.054897,3103.072402,-48.865213,728.880724,0.0
2,0.096815,6.419315,2.648174,-4.809300,1.665474,1.135428,1.978068,6.671557,-6.483499,-0.718468,...,118.276624,73.557180,-152.424573,104.723097,-0.248113,-0.433720,5248.680776,55.262809,-830.828786,0.0
3,1.353247,4.872031,2.347735,-4.659608,2.973500,-0.689366,0.820415,7.067547,-1.101120,-0.390746,...,326.647490,174.481041,-254.697019,312.415596,0.033894,-1.430820,29021.428260,-175.083707,-3405.572968,0.0
4,-0.636940,3.136887,2.225336,-5.318593,2.762497,-0.825300,0.334317,5.110205,-0.947879,0.346117,...,18.449083,18.800120,-74.725223,3.410771,-2.874054,10.718372,357.663456,27.322223,289.073617,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,-0.017573,0.625753,0.268371,-0.448639,0.189354,0.922201,0.822887,0.068731,-0.598955,0.016109,...,0.918172,0.317626,-0.529395,0.277782,0.806216,2.150479,0.096817,-0.577462,-0.350330,1.0
93,-0.017752,0.140717,0.103851,-0.238601,0.148459,-0.310604,-0.303442,0.010561,0.111476,0.027426,...,0.303191,0.125406,-0.156325,0.167704,0.210820,-0.477192,0.018249,-0.012210,-1.010923,1.0
94,0.105422,1.346866,0.486940,-0.628445,0.405242,1.217277,1.762451,0.236369,-0.516199,-0.063011,...,0.189860,0.067965,-0.063803,0.044441,1.168573,0.670551,0.006049,0.012354,-0.691943,1.0
95,0.151116,1.267792,0.646418,-1.924078,0.191489,-1.368972,5.480673,0.419800,-0.159406,-0.108240,...,2.450752,1.084747,-2.192790,0.513545,0.030822,1.533995,1.163984,-1.074096,58.514092,1.0


In [ ]:
final_test.to_csv('/content/gdrive/MyDrive/Phone Collected Test SVM.csv')